# Predicting Budds text and reformatting its schema for CharaParser
## LOG NOTEBOOK
### Nov. 27th 2019
#### TODO: Build out workflow
##### Workflow steps:

* Import text file by reading the lines
* Push the lines into the model pipeline
* Add the classifications as XML nodes
* Reformat to input schema
* Break up data into XML documents

In [3]:
# Auto update of packages within the notebook
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np

# Import custom modelling code
module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.predict_model import *
from src.features.build_length_features import locate_empty_strings

ModuleNotFoundError: No module named 'features'

##### Importing Budd's text

In [13]:
budds_file_name = "/Users/jocelynpender/flora-prepper-proj/data/external/texts/budds_testing.txt"
budds_lines = open(budds_file_name).read().splitlines()

In [ ]:
# TODO: if two key tags occur right after one another, merge the key tags into one key tag

##### Import the vectorizer and create Budd's DTM

In [20]:
custom_vec = joblib.load("../../models/custom_vec") # load the DTM vectorizer
test_dtm = custom_vec.transform(budds_lines) # fit the DTM to the new data

##### Import the classifier and generate predictions

In [22]:
# load the model and generate predictions
clf = joblib.load("../../models/classifier_model") # load the classifier
predicted = clf.predict(test_dtm) # predict the classification based on the DTM and the model
dtm_predictions_series = pd.Series(predicted)

##### Reformat the results for easy viewing

In [37]:
budds_lines = pd.Series(budds_lines)
budds_results = pd.concat([budds_lines, dtm_predictions_series], axis=1)
budds_results = budds_results[locate_empty_strings(budds_results[0])] # Remove rows with empty strings as text
budds_results.columns = ["text", "classification"] # Set column names
budds_results.to_csv(path_or_buf="budds_results_to_examine.csv")
budds_results

,0,1
0,Keys to main groups,key
1,"Key to Divisions, Subdivisions, and Classes",key
2,,morphology
3,"1. Plants without true flowers, reproducing",key
4,by spores,morphology
...,...,...
29680,3 ^073 0005Zb23 fl DATE DUE DEC,taxon_identification
29681,,morphology
29682,\ k ?(M»7 GAYLORO,taxon_identification
29683,PfllNKD IN USA.,habitat


### Dec. 17th 2019
##### Find cells that are not keys, and that are surrounded by keys on either side!

In [85]:
budds_results = pd.read_csv("budds_results_to_examine.csv", index_col=0)
budds_results.columns = ["text", "classification"]
budds_results['reclassification'] = 'blank' # Add a new columsn to modify
budds_results.reset_index(inplace = True, drop = True) # Index was not accurate anymore
budds_results

,text,classification,reclassification
0,Keys to main groups,key,blank
1,"Key to Divisions, Subdivisions, and Classes",key,blank
2,"1. Plants without true flowers, reproducing",key,blank
3,by spores,morphology,blank
4,"Plants with flowers, reproducing by seeds.",morphology,blank
...,...,...,...
28525,imPiMiFL.. 8IB LIOTHEQUE,morphology,blank
28526,AGRICULTURE CANADA OTTAWA K \ IA 0C5,habitat,blank
28527,3 ^073 0005Zb23 fl DATE DUE DEC,taxon_identification,blank
28528,\ k ?(M»7 GAYLORO,taxon_identification,blank


In [86]:
for index, row in budds_results.iterrows():
    if index > 0 and index < len(budds_results):
        if budds_results.iloc[index-1].classification == 'key' and budds_results.iloc[index+1].classification == 'key' and row.classification != 'key':
            row.reclassification = 'key'
        else:
            row.reclassification = row.classification

Working on 18 
key
key
morphology
The false index is 18 
Stems with a central pith, or if woody, the 
text                Stems with a central pith, or if woody, the 
classification                                        morphology
reclassification                                           blank
Name: 18, dtype: object
Working on 20 
key
key
morphology
The false index is 20 
layers; leaves net-veined; seeds with 
text                layers; leaves net-veined; seeds with 
classification                                  morphology
reclassification                                     blank
Name: 20, dtype: object
Working on 22 
key
key
taxon_identification
The false index is 22 
Division: 
text                          Division: 
classification      taxon_identification
reclassification                   blank
Name: 22, dtype: object
Working on 24 
key
key
taxon_identification
The false index is 24 
Division: 
text                          Division: 
classification      taxon_identificati

Working on 380 
key
key
taxon_identification
The false index is 380 
2-seeded samara ACERACEAE, p. 515 
text                2-seeded samara ACERACEAE, p. 515 
classification                    taxon_identification
reclassification                                 blank
Name: 380, dtype: object
Working on 387 
key
key
taxon_identification
The false index is 387 
(Amorpha) 
text                          (Amorpha) 
classification      taxon_identification
reclassification                   blank
Name: 387, dtype: object
Working on 393 
key
key
morphology
The false index is 393 
Stems not armed with prickles, 
text                Stems not armed with prickles, 
classification                           morphology
reclassification                              blank
Name: 393, dtype: object
Working on 401 
key
key
taxon_identification
The false index is 401 
drupe RHAMNACEAE, p. 518 
text                drupe RHAMNACEAE, p. 518 
classification           taxon_identification
reclassification   

Working on 869 
key
key
morphology
The false index is 869 
3. Stipe brown, scaly, and pubescent 4 
text                3. Stipe brown, scaly, and pubescent 4 
classification                                   morphology
reclassification                                      blank
Name: 869, dtype: object
Working on 888 
key
key
morphology
The false index is 888 
brown, stout, to 1 mm thick, scaly and pubescent, articulate below middle; 
text                brown, stout, to 1 mm thick, scaly and pubesce...
classification                                             morphology
reclassification                                                blank
Name: 888, dtype: object
Working on 930 
key
key
morphology
The false index is 930 
Stems usually not branched above the 
text                Stems usually not branched above the 
classification                                 morphology
reclassification                                    blank
Name: 930, dtype: object
Working on 940 
key
key
morpho

Working on 1523 
key
key
morphology
The false index is 1523 
with old leaf sheaths. Flowering stem 40-80 cm high, with flowers in a raceme 
text                with old leaf sheaths. Flowering stem 40-80 cm...
classification                                             morphology
reclassification                                                blank
Name: 1523, dtype: object
Working on 1547 
key
key
morphology
The false index is 1547 
the leaves, its branches generally curved 
text                the leaves, its branches generally curved 
classification                                      morphology
reclassification                                         blank
Name: 1547, dtype: object
Working on 1568 
key
key
taxon_identification
The false index is 1568 
Sagittaria cuneata Sheld. (Fig. 22) arum-leaved arrowhead 
text                Sagittaria cuneata Sheld. (Fig. 22) arum-leave...
classification                                   taxon_identification
reclassification                   

Working on 2110 
key
key
morphology
The false index is 2110 
Culms usually decumbent, geniculate; 
text                Culms usually decumbent, geniculate; 
classification                                 morphology
reclassification                                    blank
Name: 2110, dtype: object
Working on 2194 
key
key
morphology
The false index is 2194 
Spikelets mostly 3-flowered; awn stout, 
text                Spikelets mostly 3-flowered; awn stout, 
classification                                    morphology
reclassification                                       blank
Name: 2194, dtype: object
Working on 2221 
key
key
morphology
The false index is 2221 
scattered hairs. Spikes 35-50, usually pendulous, secund; spikelets 5-8 in a 
text                scattered hairs. Spikes 35-50, usually pendulo...
classification                                             morphology
reclassification                                                blank
Name: 2221, dtype: object
Working on 2250

...

Name: 22043, dtype: object
Working on 22069 
key
key
morphology
The false index is 22069 
racemose Composed of or resembling racemes. 
text                racemose Composed of or resembling racemes. 
classification                                        morphology
reclassification                                           blank
Name: 22069, dtype: object
Working on 22072 
key
key
morphology
The false index is 22072 
radiate Having ray florets (Fig. 199, p. 684). 
text                radiate Having ray florets (Fig. 199, p. 684). 
classification                                           morphology
reclassification                                              blank
Name: 22072, dtype: object
Working on 22094 
key
key
morphology
The false index is 22094 
sac A pouch, especially the cavity of an anther. 
text                sac A pouch, especially the cavity of an anther. 
classification                                             morphology
reclassification                                

Working on 23454 
key
key
taxon_identification
The false index is 23454 
ground-ivy 623 
text                     ground-ivy 623 
classification      taxon_identification
reclassification                   blank
Name: 23454, dtype: object
Working on 23468 
key
key
taxon_identification
The false index is 23468 
mountain 117 
text                       mountain 117 
classification      taxon_identification
reclassification                   blank
Name: 23468, dtype: object
Working on 23520 
key
key
habitat
The false index is 23520 
hemlock 553 
text                hemlock 553 
classification           habitat
reclassification           blank
Name: 23520, dtype: object
Working on 23563 
key
key
habitat
The false index is 23563 
woodland 50 
text                woodland 50 
classification           habitat
reclassification           blank
Name: 23563, dtype: object
Working on 23604 
key
key
morphology
The false index is 23604 
erect 317 
text                erect 317 
classification      m

Working on 24859 
key
key
taxon_identification
The false index is 24859 
waterwort 525 
text                      waterwort 525 
classification      taxon_identification
reclassification                   blank
Name: 24859, dtype: object
Working on 24870 
key
key
taxon_identification
The false index is 24870 
northern 87 
text                        northern 87 
classification      taxon_identification
reclassification                   blank
Name: 24870, dtype: object
Working on 24884 
key
key
taxon_identification
The false index is 24884 
thick-leaved 404 
text                   thick-leaved 404 
classification      taxon_identification
reclassification                   blank
Name: 24884, dtype: object
Working on 24886 
key
key
taxon_identification
The false index is 24886 
whitlowwort 354 
text                    whitlowwort 354 
classification      taxon_identification
reclassification                   blank
Name: 24886, dtype: object
Working on 24924 
key
key
taxon_identificatio

Working on 26155 
key
key
taxon_identification
The false index is 26155 
atropurpurea 117 
text                   atropurpurea 117 
classification      taxon_identification
reclassification                   blank
Name: 26155, dtype: object
Working on 26157 
key
key
taxon_identification
The false index is 26157 
danthonioides 117 
text                  danthonioides 117 
classification      taxon_identification
reclassification                   blank
Name: 26157, dtype: object
Working on 26186 
key
key
habitat
The false index is 26186 
Disporum 257, 260 
text                Disporum 257, 260 
classification                 habitat
reclassification                 blank
Name: 26186, dtype: object
Working on 26243 
key
key
taxon_identification
The false index is 26243 
austriaca 41 
text                       austriaca 41 
classification      taxon_identification
reclassification                   blank
Name: 26243, dtype: object
Working on 26245 
key
key
taxon_identification
The false 

Working on 28192 
key
key
taxon_identification
The false index is 28192 
Sparganium 63 
text                      Sparganium 63 
classification      taxon_identification
reclassification                   blank
Name: 28192, dtype: object
Working on 28194 
key
key
taxon_identification
The false index is 28194 
chlorocarpum var. acaule 63 
text                chlorocarpum var. acaule 63 
classification              taxon_identification
reclassification                           blank
Name: 28194, dtype: object
Working on 28286 
key
key
habitat
The false index is 28286 
Taraxacum 689, 702, 804 
text                Taraxacum 689, 702, 804 
classification                       habitat
reclassification                       blank
Name: 28286, dtype: object
Working on 28307 
key
key
taxon_identification
The false index is 28307 
var. venulosum 382 
text                 var. venulosum 382 
classification      taxon_identification
reclassification                   blank
Name: 28307, dtype: obj

In [4]:
# budds_results.to_csv(path_or_buf="budds_results_to_examine_rekey.csv")
budds_results = pd.read_csv("budds_results_to_examine_rekey.csv", index_col=0)
budds_results

,text,classification,reclassification
0,Keys to main groups,key,blank
1,"Key to Divisions, Subdivisions, and Classes",key,key
2,"1. Plants without true flowers, reproducing",key,key
3,by spores,morphology,morphology
4,"Plants with flowers, reproducing by seeds.",morphology,morphology
...,...,...,...
28525,imPiMiFL.. 8IB LIOTHEQUE,morphology,morphology
28526,AGRICULTURE CANADA OTTAWA K \ IA 0C5,habitat,habitat
28527,3 ^073 0005Zb23 fl DATE DUE DEC,taxon_identification,taxon_identification
28528,\ k ?(M»7 GAYLORO,taxon_identification,taxon_identification


##### Add XML to classified tags

In [5]:
# schema tags for budds classification:
# TODO: find schema document from Github

budds_results.reclassification[0] = budds_results.classification[0] # Fix first item

In [6]:
# https://stackoverflow.com/questions/14358567/finding-consecutive-segments-in-a-pandas-data-frame
index_matrix = budds_results.reset_index().groupby('reclassification')['index'].apply(np.array) # Find sequences of classifications
#index_matrix['key']

budds_results['block'] = (budds_results.reclassification.shift(1) != budds_results.reclassification).astype(int).cumsum()
budds_results

In [8]:
# Take the block, paste the text all together, and wrap it in the appropriate XML tag
# Build a dictionary of tag

# <taxon_identification status="ACCEPTED"></taxon_identification>
# <description type="morphology"></description>
# <description type="habitat"></description>
# <key></key>
open_tags = {'taxon_identification': '<taxon_identification status="ACCEPTED">',
             'morphology': '<description type="morphology">',
             'habitat': '<description type="habitat">',
             'key': '<key>'
            }
close_tags = {'taxon_identification': '</taxon_identification>',
              'morphology': '</description>',
              'habitat': '</description>',
              'key': '</key>'
    
}

In [30]:
group_blocks = budds_results.groupby('block')
runs = group_blocks['text'].apply(np.array)
runs

block
1        [Keys to main groups , Key to Divisions, Subdi...
2        [by spores , Plants with flowers, reproducing ...
3              [2. Plants bearing flowers having neither ]
4        [styles nor stigmas, the naked ovules , (seeds...
5        [bearing cones or cone-like fruit , Plants bea...
                               ...                        
12346                                                 [> ]
12347                          [imPiMiFL.. 8IB LIOTHEQUE ]
12348              [AGRICULTURE CANADA OTTAWA K \ IA 0C5 ]
12349    [3 ^073 0005Zb23 fl DATE DUE DEC , \ k ?(M»7 G...
12350                                   [PfllNKD IN USA. ]
Name: text, Length: 12350, dtype: object

### Dec. 18th 2019

In [31]:
#'\n'.join(runs.iloc[0])
strings = runs.apply(lambda x: '\n'.join(x))
strings

block
1        Keys to main groups \nKey to Divisions, Subdiv...
2        by spores \nPlants with flowers, reproducing b...
3                2. Plants bearing flowers having neither 
4        styles nor stigmas, the naked ovules \n(seeds)...
5        bearing cones or cone-like fruit \nPlants bear...
                               ...                        
12346                                                   > 
12347                            imPiMiFL.. 8IB LIOTHEQUE 
12348                AGRICULTURE CANADA OTTAWA K \ IA 0C5 
12349    3 ^073 0005Zb23 fl DATE DUE DEC \n\ k ?(M»7 GA...
12350                                     PfllNKD IN USA. 
Name: text, Length: 12350, dtype: object

In [32]:
group_blocks['reclassification'].take([0])

block       
1      0                         key
2      3                  morphology
3      5                         key
4      6                  morphology
5      9                         key
                        ...         
12346  28524                     key
12347  28525              morphology
12348  28526                 habitat
12349  28527    taxon_identification
12350  28529                 habitat
Name: reclassification, Length: 12350, dtype: object

In [33]:
index_matrix

reclassification
habitat                 [560, 564, 568, 572, 583, 589, 655, 661, 669, ...
key                     [0, 1, 2, 5, 9, 10, 14, 15, 16, 17, 18, 19, 20...
morphology              [3, 4, 6, 7, 8, 11, 12, 13, 60, 61, 73, 110, 1...
taxon_identification    [32, 33, 72, 109, 128, 138, 189, 190, 206, 225...
Name: index, dtype: object